In [1]:
import pandas as pd
import os
import json

from pprint import pprint

import chart_studio.plotly as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
import random
init_notebook_mode(connected=True)

In [6]:
def plot_3x3_grid(run_dir_path):
    # Gather files from combined firing optimisation
    files = list(map(lambda x: f'{run_dir_path}/{x}', os.listdir(run_dir_path)))

    file_firing = []

    for filename in files:
        params = json.load(open(f'{filename}/params.json'))
        
        file_firing.append((
            filename,
            params,
            pd.read_csv(f'{filename}/frontier.csv'),
            pd.read_csv(f'{filename}/opt.csv')
        ))
      
    # Sort results by sparsity, excitatory_firing, inhibitory firing
    firings = sorted(
        file_firing, 
        key=lambda x: (-x[1]['sparsity'],-x[1]['ex_target'],x[1]['in_target'])
    )

    plot_titles = [
        f'f: {params["sparsity"]}, exc: {params["ex_target"]}, inh: {params["in_target"]}'
        for filename, params, frontier, opt in firings
    ]

    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=plot_titles,
        x_title='Excitatory neural firing rate error (Hz)',
        y_title='Inhibitory neural firing rate error (Hz)',
    )

    positions =[
        (1,1),(1,2),(1,3),
        (2,1),(2,2),(2,3),
        (3,1),(3,2),(3,3) 
    ]

    for i, (filename, params, frontier, opt) in enumerate(firings, start=1):
        position = positions[i-1]

        fig.add_trace(go.Scatter(
                x=frontier['inhib_firing_error'],
                y=frontier['ex_firing_error'],
                name='excitatory connections',
                mode='markers',
                marker={
                    'color':'black'
                },
            ), 
            row=position[0], col=position[1]
        )

    fig.update_layout(
        height=900, width=900, title='Final frontiers of firing error',

    )


    fig.show()

# Multi-Objective Optimisation of Spiking Neural Networks

## Pareto front convergence

## Constrained search
Attempted to reduce errors by reducing the set of solutions that are accepted into the frontier. With a fixed max error of 5 the search has much hight errors for all combanations, and finds no possible solutions with a firing rate of 10,2.

In [11]:
plot_3x3_grid('/Users/fitzgerald/thesis/src/results/constrained-firing/constraint')

## Combined firing optimisation
Attempted to reduce errors by minimising the sum of the errors
$ r_3 = r_1 + r_2 $

In [12]:
plot_3x3_grid('/Users/fitzgerald/thesis/src/results/combined')

## Sum squared errors
An attempt to minimse the distance from the origin $ r_3 = \sqrt{r_1^2 + r_2^2} $

In [14]:
plot_3x3_grid('/Users/fitzgerald/thesis/src/results/squared_error')

## Time bin firing optimisation
Oo=